In [1]:
import face_model
import argparse
import cv2
import sys
import numpy as np
import time
import os
from os import listdir
import pickle
import random
import matplotlib.pyplot as plt
import insightface
from insightface.utils import face_align

C:\Users\lalit\anaconda3\envs\base2\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\lalit\anaconda3\envs\base2\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
C:\Users\lalit\anaconda3\envs\base2\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
model = face_model.FaceModel(0,'model-r100-ii/model', 0)

[32, 16, 8] {'32': {'SCALES': (32, 16), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}, '16': {'SCALES': (8, 4), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}, '8': {'SCALES': (2, 1), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}}
use_landmarks True
loading model-r100-ii/model 0


In [3]:
import insightface
from insightface.utils import face_align
detector = insightface.model_zoo.get_model('retinaface_mnet025_v2')
detector.prepare(ctx_id=0)

[32, 16, 8] {'32': {'SCALES': (32, 16), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}, '16': {'SCALES': (8, 4), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}, '8': {'SCALES': (2, 1), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}}
use_landmarks True


In [4]:
Embedding_and_ID = pickle.load(open('Embedding_and_ID.pkl','rb'))
Name_and_ID = pickle.load(open('Name_and_ID.pkl','rb'))

In [5]:
def match_face(image_address):
    t0 = time.time()
    image = cv2.imread(image_address)
    bbox, pts5 = detector.detect(image, threshold=0.8)
    for i in range(len(bbox)):
        x1, y1, x2, y2 = bbox[i,0:4]
        x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
        nimg = face_align.norm_crop(image, pts5[i])
        embd = model.get_feature(nimg)
        cv2.rectangle(image, (x1,y1), (x2,y2), (0,0,255), 1)
        max_sim = {}
        for j in Embedding_and_ID:
            representation = Embedding_and_ID[j]
            similarity = np.dot(embd, representation)
            if similarity>0.4:
                max_sim[j] = similarity
        if len(max_sim)!=0:
            max_val = max(max_sim.values())
            for key, value in max_sim.items():
                if max_val == value:
                    ID = key
            person_name = Name_and_ID[ID]
            print(max_sim[ID])
            print(person_name)
            cv2.putText(image, person_name, (x1, y1-2), 0, 0.5, (0, 0, 255), 2)
        else:
            cv2.putText(image, 'unknown', (x1, y1-5), 0, 0.6, (255, 0, 0), 2)
    cv2.imwrite('save images/'+str(22)+ '.jpg',image)
    t4 = time.time()
    print(t4-t0,'sec')
    cv2.imshow('Face Recognition', image)
    cv2.waitKey(0)    
    cv2.destroyAllWindows()

In [10]:
match_face('test images/138.jpg')

0.7430223
Narendra_Modi
0.7892616
Yogi_Adityanath
0.77564627
Amit_Shah
0.7535145
Manohar Parrikar
4.114994525909424 sec


In [14]:
def face_match2(frame):
    image = frame
    bbox, pts5 = detector.detect(frame, threshold=0.8)
    for i in range(len(bbox)):
        x1, y1, x2, y2 = bbox[i,0:4]
        x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
        nimg = face_align.norm_crop(image, pts5[i])
        embd = model.get_feature(nimg)
        cv2.rectangle(image, (x1,y1), (x2,y2), (0,0,255), 1)
        max_sim = {}
        for j in Embedding_and_ID:
            representation = Embedding_and_ID[j]
            similarity = np.dot(embd, representation)
            if similarity>0.4:
                max_sim[j] = similarity
        if len(max_sim)!=0:
            max_val = max(max_sim.values())
            for key, value in max_sim.items():
                if max_val == value:
                    ID = key
            person_name = Name_and_ID[ID]
            print(max_sim[ID])
            print(person_name)
            cv2.putText(image, person_name, (x1, y1-2), 0, 0.5, (0, 0, 255), 2)
        else:
            cv2.putText(image, 'unknown', (x1, y1-5), 0, 0.6, (255, 0, 0), 2)
    if image.shape[1]> 400:
        im2 = cv2.resize(image,(int(image.shape[1]*0.6),int(image.shape[0]*0.6)))
    return im2

In [15]:
def face_recognition_vid(video_address):
    filename = video_address
    dir_name = os.path.splitext(filename)[0]
    parent_dir = "video output/"
    directory = os.path.join(parent_dir, dir_name[12:]) 
    os.mkdir(directory)
    seconds = 1
    cap = cv2.VideoCapture(filename)

    # Get the frames per second
    fps = round(cap.get(cv2.CAP_PROP_FPS)) 
    multiplier = fps*seconds

    # Get the total numer of frames in the video.
    frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    print("FC :", frame_count)

    count = -1

    while cap.isOpened():
        ret, frame = cap.read()

        if ret:
            frame2 = face_match2(frame)
            #cv2.imwrite('F:/Output/%s/Frame{:f}.jpg'.format(count),frame)
            cv2.imwrite('video output/%s/image{:f}.jpg'.format(count)%(dir_name[12:]),frame2)
            count += multiplier 
            cap.set(1, count)
        else:
            cap.release()
            break

In [10]:
face_recognition_vid('test videos/test_video4.mp4')

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'video output/test_video4'

In [16]:
def match_face2(frame):
    image = frame
    bbox, pts5 = detector.detect(frame, threshold=0.8)
    for i in range(len(bbox)):
        x1, y1, x2, y2 = bbox[i,0:4]
        x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
        nimg = face_align.norm_crop(image, pts5[i])
        embd = model.get_feature(nimg)
        cv2.rectangle(image, (x1,y1), (x2,y2), (0,0,255), 1)
        max_sim = {}
        for j in Embedding_and_ID:
            representation = Embedding_and_ID[j]
            similarity = np.dot(embd, representation)
            if similarity>0.43:
                max_sim[j] = similarity
        if len(max_sim)!=0:
            max_val = max(max_sim.values())
            for key, value in max_sim.items():
                if max_val == value:
                    ID = key
            person_name = Name_and_ID[ID]
            cv2.putText(image, person_name, (x1, y1-2), 0, 0.5, (0, 0, 255), 2)
        else:
            cv2.putText(image, 'unknown', (x1, y1-5), 0, 0.6, (255, 0, 0), 2)
    return image

In [17]:
cap = cv2.VideoCapture(0)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('video output2/fr_webcam2.mp4',fourcc,15.0,(640,480))
start_time = time.time()
fps = 0
total_frames = 0
while (cap.isOpened()):
    ret, img = cap.read()
    if not ret:
        break
    total_frames = total_frames+1
    end_time = time.time()
    time_diff= end_time-start_time
    if time_diff ==0:
        fps = 0.0
    else:
        fps = (total_frames/time_diff)
    fps_text =  "FPS: {:.2f}".format(fps)
    img = match_face2(img)
    cv2.putText(img, fps_text, (5,30), 0, 0.6, (100, 100, 255), 2)
    img = cv2.resize(img,(640,480))
    out.write(img)
    cv2.imshow('face',img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break 
cap.release()
out.release()
cv2.destroyAllWindows()

In [102]:
cap = cv2.VideoCapture('test videos/test_video1.mp4')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('video output2/fr_test_video7.mp4',fourcc,11.0,(480,620))
i = 0
start_time = time.time()
fps = 0
total_frames = 0
while True:
    ret, img = cap.read()
    if not ret:
        break
    if i%2 ==0:
        img = match_face2(img)
        total_frames = total_frames+1
        end_time = time.time()
        time_diff= end_time-start_time
        if time_diff ==0:
            fps = 0.0
        else:
            fps = (total_frames/time_diff)
        fps_text =  "FPS: {:.2f}".format(fps)
        cv2.putText(img, fps_text, (5,30), 0, 0.6, (100, 100, 255), 2)
        img = cv2.resize(img,(480,620))
        out.write(img)
        cv2.imshow('face',img)
    i = i+1
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break 
cap.release()
cv2.destroyAllWindows()

In [65]:
cap = cv2.VideoCapture('test videos/test_video1.mp4')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('subscribe1.mp4',fourcc,25.0,(640,480))
while(cap.isOpened()):
    ret, img = cap.read()
    if not ret:
        break
    img = cv2.resize(img,(640,480))
    out.write(img)
    cv2.imshow('face',img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break 
cap.release()
out.release()
cv2.destroyAllWindows()

In [22]:
cap = cv2.VideoCapture(0)
start_time = time.time()
fps = 0
total_frames = 0
while (cap.isOpened()):
    ret, img = cap.read()
    if not ret:
        break
    total_frames = total_frames+1
    end_time = time.time()
    time_diff= end_time-start_time
    if time_diff ==0:
        fps = 0.0
    else:
        fps = (total_frames/time_diff)
    fps_text =  "FPS: {:.2f}".format(fps)
    img = match_face2(img)
    cv2.putText(img, fps_text, (5,30), 0, 0.6, (100, 100, 255), 2)
    img = cv2.resize(img,(640,480))
    cv2.imshow('face',img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break 
cap.release()
out.release()
cv2.destroyAllWindows()

In [15]:
cap = cv2.VideoCapture('test videos/test_video1.mp4')
i = 0
start_time = time.time()
fps = 0
total_frames = 0
while True:
    ret, img = cap.read()
    if not ret:
        break
    if i%2 ==0:
        img = match_face2(img)
        total_frames = total_frames+1
        end_time = time.time()
        time_diff= end_time-start_time
        if time_diff ==0:
            fps = 0.0
        else:
            fps = (total_frames/time_diff)
        fps_text =  "FPS: {:.2f}".format(fps)
        cv2.putText(img, fps_text, (5,30), 0, 0.6, (100, 100, 255), 2)
        cv2.imshow('face',img)
    i = i+1
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break 
cap.release()
cv2.destroyAllWindows()

In [3]:
import h5py

In [4]:
h5py.__version__

'2.10.0'